# Exercise08 : Publish as a Web Service

Finally we publish our model as a web service.

Before running this code, **complete the model registration in "[Exercise04 : Train on Remote GPU Virtual Machine](./exercise04_train_remote.ipynb)"**.

*back to [index](https://github.com/tsmatz/azureml-tutorial/)*

## Variable's Setting

Replace below's branket's string and set the required variables.

In [1]:
my_resource_group = "{AML-RESOURCE-GROUP-NAME}"
my_workspace = "{AML-WORSPACE-NAME}"

## Create entry script (.py)

In order to deploy as web service, first we generate the following scoring code.<br>
This entry script in AML should include both ```init()``` and ```run()```.

> Note : The serving compute (VM) provides [managed identity endpoint](https://docs.microsoft.com/en-us/azure/active-directory/managed-identities-azure-resources/how-to-use-vm-token). (Your script can use both system-assigned identity and user-assigned identity.) Your script can then get the access permissions for Azure resources without providing secure information.

In [2]:
%%writefile score.py
import os
import json
import tensorflow as tf
from azureml.core.model import Model

def init():
    global pred_fn
    ## model_path = Model.get_model_path(model_name='mnist_model_test')
    model_path = os.path.join(
        os.getenv("AZUREML_MODEL_DIR"), "generated_model"
    )
    pred_fn = tf.contrib.predictor.from_saved_model(model_path)

def run(raw_data):
    try:
       data = json.loads(raw_data)['data']
       result = pred_fn({'inputs': data})
       return result['classes'].tolist()
    except Exception as e:
       result = str(e)
       return 'Internal Exception : ' + result

Writing score.py


## Create a managed endpoint

There exist **endpoint** and **deployment** in deployment topology in managed online endpoint.<br>
You can run multiple deployments in a single endpoint, and allocate appropriate traffic for these multiple deployments.

First, create a managed endpoint for deployment target.

In [3]:
%%writefile 08_managed_endpoint.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: my-mnist-endpoint
auth_mode: key

Writing 08_managed_endpoint.yml


In [4]:
!az ml online-endpoint create --file 08_managed_endpoint.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "auth_mode": "key",
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AzureML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/onlineEndpoints/my-mnist-endpoint",
  "identity": {
    "principal_id": "8936e704-6573-4eb2-8df5-aa5bde220299",
    "tenant_id": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "type": "system_assigned"
  },
  "location": "eastus",
  "name": "my-mnist-endpoint",
  "properties": {
    "AzureAsyncOperationUri": "https://management.azure.com/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:dd33cef1-2535-4c5b-947a-0085dbd5db07:f17aaac1-e14f-4ba0-9d29-8ba305d15e9c?api-version=2021-10-01",
    "azureml.onlineendpointid": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/azureml-rg/providers/micros

## Deploy as web service

Next we deploy the serving code (```score.py```) as a web service in the previous endpoint.

Before deployment, create conda configuration for serving environment.

In [5]:
%%writefile 08_conda_env.yml
name: serving_example
dependencies:
- python=3.6
- pip:
  - azureml-defaults
- tensorflow==1.15
channels:
- anaconda
- conda-forge

Writing 08_conda_env.yml


Now we deploy a web service with yaml configuration for deployment.

When you change the model (or code) in managed endpoint, you can submit multiple deployments and transfer the traffic allocation without causing any disruption.<br>
With the following ```--all-traffic``` option, all traffic (100% traffic) will be allocated to this single deployment.

In this example, I use the trained model in Exercise04, and **run "[Exercise04 : Train on Remote GPU Virtual Machine](./exercise04_train_remote.ipynb)", before running this code.**

> Note : You can scale computes by increasing the following ```instance_count```. (You can also define auto-scale settings.)

In [6]:
%%writefile 08_managed_deployment.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: my-mnist-deployment-v1
endpoint_name: my-mnist-endpoint
model: azureml:mnist_model_test:1
code_configuration:
  code: 
    local_path: ./
  scoring_script: score.py
environment: 
  conda_file: 08_conda_env.yml
  image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
instance_type: Standard_DS2_v2
instance_count: 1

Writing 08_managed_deployment.yml


In [10]:
!az ml online-deployment create --file 08_managed_deployment.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --all-traffic

Command group 'ml online-deployment' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Check: endpoint my-mnist-endpoint exists
Uploading cli_yaml (61.18 MBs): 100%|█| 61180104/61180104 [00:01<00:00, 50106038


The deployment request ws01-my-mnist-endpoint-6872361 was accepted. ARM deployment URI for reference: 
https://ms.portal.azure.com/#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2Fb3ae1c15-4fef-4362-8c3a-5d804cdeb18d%2FresourceGroups%2FAzureML-rg%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Fws01-my-mnist-endpoint-6872361
Registering environment version: (b562c3a6-a6a5-47a7-a5dd-cf96170c281c 1 )  Done (2s)
Registering code version: (cd4bb3ef-2e9e-45f6-858a-421eb6b8dcb9 1 )  Done (0s)
Creating or updating deployment: my-mnist-deployment-v1   .......................................................................  Done (6m 10s)
Total time : 6m 12s
{
  "app_insights_enabled": false,
  "code_configuration": 

Please see logs as follows, if error has occured.<br>
For instance, the following log output shows Python import error in entry script.

In [9]:
!az ml online-deployment get-logs --endpoint-name my-mnist-endpoint \
  --name my-mnist-deployment-v1 \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml online-deployment' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Instance status:
SystemSetup: Succeeded
UserContainerImagePull: Succeeded
ModelDownload: Succeeded
UserContainerStart: InProgress

Container logs:
2022-02-28T07:19:01,824500903+00:00 - rsyslog/run 
2022-02-28T07:19:01,827055025+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-02-28T07:19:01,844367081+00:00 - nginx/run 
2022-02-28T07:19:01,856197687+00:00 - iot-server/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-02-28T07:19:02,048559714+00:00 - iot-server/finish 1 0
2022-02-28T07:19:02,050840535+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (12)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 37
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2022-

> Note : Before submitting a deployment on cloud, you can submit and debug your deployment on local docker runtime. (See [here](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-managed-online-endpoints).)<br>
> With Visual Studio Code, you can also attach debugger on local deployment.

## Test your web service

Let's invoke your deployed web service and check the returned results in Python.

First see URI (address) for your deployed web service.

In [12]:
!az ml online-endpoint show --name my-mnist-endpoint \
  --query scoring_uri \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
"https://my-mnist-endpoint.eastus.inference.ml.azure.com/score"


Extract key credential for this endpoint.

In [13]:
!az ml online-endpoint get-credentials \
  --name my-mnist-endpoint \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "primaryKey": "3cMIeMz6TJQlSqrIVxeZXRFBaGrEFhdq",
  "secondaryKey": "S5NEF60bf6nlqLoPcKnrnBttILZaxSu6"
}


Now let's invoke scoring web service in Python.<br>
(**Replace the following ```SERVING_URI``` and ```API_KEY``` with yours**.)

In [15]:
import requests
import json

import tensorflow as tf

SERVING_URI = "https://my-mnist-endpoint.eastus.inference.ml.azure.com/score"
API_KEY = "3cMIeMz6TJQlSqrIVxeZXRFBaGrEFhdq"

# Read data by tensor
tfdata = tf.data.TFRecordDataset('./data/test.tfrecords')
iterator = tf.compat.v1.data.make_one_shot_iterator(tfdata)
data_org = iterator.get_next()
###
data_exam = tf.parse_single_example(
    data_org,
    features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64)
    })
data_image = tf.decode_raw(data_exam['image_raw'], tf.uint8)
data_image.set_shape([784])
data_image = tf.cast(data_image, tf.float32) * (1. / 255)
data_label = tf.cast(data_exam['label'], tf.int32)

# Run tensor and generate data
with tf.Session() as sess:
    image_arr = []
    label_arr = []
    for i in range(3):
        image, label = sess.run([data_image, data_label])
        image_arr.append(image.tolist())
        label_arr.append(label)

# Invoke web service !
headers = {
    'Content-Type':'application/json',
    'Authorization':('Bearer '+ API_KEY)
} 
values = json.dumps(image_arr)
input_data = "{\"data\": " + values + "}"
http_res = requests.post(
    SERVING_URI,
    input_data,
    headers = headers)
print('Predicted : ', http_res.text)
print('Actual    : ', label_arr)

Predicted :  [7, 2, 1]
Actual    :  [7, 2, 1]


## Remove endpoint

In [16]:
!az ml online-endpoint delete --name my-mnist-endpoint \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --yes

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Deleting endpoint my-mnist-endpoint ........................................................................Done (6m 8s)
